In [1]:
import pandas as pd

df = pd.read_csv('data/car-sales.csv').drop(columns=['Unnamed: 0'], axis=1)
df.head()

,price,sold,models_age,km_per_year
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [6]:
import numpy as np
from sklearn.model_selection import train_test_split

# Separating into labels and features
y = df['sold']
x = df[['price', 'models_age', 'km_per_year']]

# Separating the training and testing sets
SEED = 158020
np.random.seed(SEED)
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.25, stratify=y)

print('We will train with {} elements and train with {} elements.'.format(len(train_x), len(test_x)))

We will train with 7500 elements and train with 2500 elements.


First let's create a dummy classifier to compare to our model

In [3]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(train_x, train_y)
accuracy = dummy_stratified.score(test_x, test_y) * 100

print("The dummy's accuracy is {:.1f}%".format(accuracy))


The dummy's accuracy is 58.0%


Now creating our model

TO understand how the Decision Tree Classifier works, here's a very good video explaining it: https://www.youtube.com/watch?v=ZVR2Way4nwQ

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

SEED = 158020
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
model.fit(train_x, train_y)
predictions = model.predict(test_x)

accuracy = accuracy_score(test_y, predictions) * 100

print("The model's accuracy is {:.2f}%".format(accuracy))

The model's accuracy is 71.92%


However, what happens if we change our seed?

In [5]:
SEED = 5 # Changing our seed to another number
np.random.seed(SEED)

# Spliting our data into training and testing sets
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.25, stratify=y)

print('We will train with {} elements and train with {} elements.'.format(len(train_x), len(test_y)))

# Creating our model (Tree Classifier) and training it
model = DecisionTreeClassifier(max_depth=2)
model.fit(train_x, train_y)

# Making predictions with our model
predictions = model.predict(test_x)

accuracy = accuracy_score(test_y, predictions) * 100

print("The model's accuracy is {:.2f}%".format(accuracy))

We will train with 7500 elements and train with 2500 elements.
The model's accuracy is 76.84%


Look how it varied drastically! If, for example, our baseline for a good model was 75%, simply changing the seed would mke the cut! We can't make important decisions based on randomness, so we must try to minimize its effects.

## Cross Validation

A good approach is to split our data into training and testing sets various times and in different places. The more we do this, the more we minimize the effects of randomness.

In [16]:
from sklearn.model_selection import cross_validate

SEED = 158020 # Changing our seed to another number
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=3) # cv=3 means we're separating our dataset into 3 subsets and in each iteration using one of them as the test set
results['test_score']

array([0.75704859, 0.7629763 , 0.75337534])

In [19]:
mean = results['test_score'].mean()
std = results['test_score'].std()

print("Accuracy with cross validation (3) -> [%.2f, %.2f]" % ((mean - 2*std) * 100, (mean + 2*std) * 100))

Accuracy with cross validation (3) -> [74.99, 76.57]


Note that if we change the seed value, our accuracy will remain consistent!

What about the value 3 we chose for spliting our data? What happens if we choose another number?

In [21]:
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=10)

mean = results['test_score'].mean()
std = results['test_score'].std()

print("Accuracy with cross validation (10) -> [%.2f, %.2f]" % ((mean - 2*std) * 100, (mean + 2*std) * 100))

Accuracy with cross validation (10) -> [74.24, 77.32]


Some scientific papers show that choosing a cv between 5 and 10 often results in good accuracy, so we'll be choosing 5 

In [30]:
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=5)

mean = results['test_score'].mean()
std = results['test_score'].std()

print("Accuracy with cross validation (5) -> [%.2f, %.2f]" % ((mean - 2*std) * 100, (mean + 2*std) * 100))

Accuracy with cross validation (5) -> [75.21, 76.35]


However, *cross_validate()* is deterministic, and once again it is advantageous to test with random spliting in order to have a better grasp of our model's true accuracy

## Randomness in Cross Validation

In [46]:
# First of all let's create a function to print our results
def print_results(results):
    mean = results['test_score'].mean()
    std = results['test_score'].std()

    print("Mean accuracy -> %.2f" % (mean * 100))
    print("Accuracy Interval -> [%.2f, %.2f]" % ((mean - 2*std) * 100, (mean + 2*std) * 100))

In [47]:
from sklearn.model_selection import KFold

SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True) # Shuffles the data before splitting into batches
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv = cv)
print_results(results)

Mean accuracy -> 75.76
Accuracy Interval -> [73.26, 78.26]
